### portmy database: stocks table
### stock database: buy, price tables

In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()
conmy = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

today = date.today()
week_str = today.strftime('%Y-%U')
today, week_str

(datetime.date(2022, 3, 26), '2022-12')

In [2]:
today = today - timedelta(days=1)
wkago = today - timedelta(days=7)
today, wkago

(datetime.date(2022, 3, 25), datetime.date(2022, 3, 18))

### Restart & Run All Cells

### This process affects only already in port stocks. To highlight price changes in percent.

In [3]:
cols = 'name price_d price_w percent perform mrkt '.split()

In [4]:
format_dict = {
    'qty':'{:,}',    
    'price':'{:.2f}','price_d':'{:.2f}','price_w':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'maxp':'{:.2f}','minp':'{:.2f}','opnp':'{:.2f}',    
    'pct':'{:,.2f}%','percent':'{:,.2f}%',
    
    'pe':'{:.2f}','pbv':'{:.2f}',
    'paid_up':'{:,.2f}','market_cap':'{:,.2f}',
    'daily_volume':'{:,.2f}','beta':'{:,.2f}', 
    'dly_vol':'{:,.2f}', 
}

In [5]:
sql = """
SELECT name, date, price 
FROM buy 
ORDER BY date DESC 
LIMIT 1
"""
df_buy = pd.read_sql(sql, const)
df_buy.style.format(format_dict)

,name,date,price
0,AIMIRT,2022-03-10,13.00


In [6]:
names = df_buy['name']
name = names.to_string(index=False)

sql = '''
SELECT * FROM stocks WHERE name = '%s'
'''
sql = sql % name
print(sql)

df_stocks = pd.read_sql(sql, conmy)
df_stocks.style.format(format_dict) 


SELECT * FROM stocks WHERE name = 'AIMIRT'



,id,name,market,price,max_price,min_price,pe,pbv,paid_up,market_cap,daily_volume,beta,ticker_id,created_at,updated_at
0,667,AIMIRT,SET,12.80,13.40,11.71,999.99,1.06,"5,954.81","7,622.15",3.13,0.03,668,2018-06-21 02:41:32.576617,2022-03-25 20:09:56.240578


In [7]:
sql = '''
SELECT * FROM price WHERE name = "%s" ORDER BY date DESC LIMIT 5
'''
sql = sql % name
print(sql)

df_price = pd.read_sql(sql, const)
df_price.style.format(format_dict)


SELECT * FROM price WHERE name = "AIMIRT" ORDER BY date DESC LIMIT 5



,name,date,price,maxp,minp,qty,opnp
0,AIMIRT,2022-03-25,12.80,12.90,12.80,"55,900",12.80
1,AIMIRT,2022-03-24,12.80,12.90,12.80,"11,200",12.80
2,AIMIRT,2022-03-23,12.70,12.80,12.70,"26,900",12.80
3,AIMIRT,2022-03-22,12.80,12.90,12.80,"7,000",12.90
4,AIMIRT,2022-03-21,12.90,12.90,12.70,"60,825",12.70


In [8]:
sql = '''
SELECT name
FROM buy 
WHERE active = 1
ORDER BY name'''
df_price = pd.read_sql(sql, const)

names = df_price.name.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))

sql = """
SELECT name, price AS price_d 
FROM price 
WHERE date = '%s' AND name IN (%s)
ORDER BY name, date"""
sql = sql % (today, in_p)
print(sql)

df_today = pd.read_sql(sql, const)
df_today.shape[0]


SELECT name, price AS price_d 
FROM price 
WHERE date = '2022-03-25' AND name IN ('AIMIRT', 'ASP', 'BCH', 'CPNCG', 'DIF', 'DOHOME', 'HREIT', 'IP', 'JASIF', 'KCE', 'MAKRO', 'MCS', 'NER', 'PTT', 'PTTGC', 'RATCH', 'RCL', 'RJH', 'SCC', 'SCCC', 'SINGER', 'SIS', 'SPALI', 'STA', 'SYNEX', 'TISCO', 'TMT', 'TPIPL', 'WHART')
ORDER BY name, date


29

In [9]:
sql = """
SELECT name, price AS price_w
FROM price 
WHERE date = '%s' AND name IN (%s) 
ORDER BY name"""
sql = sql % (wkago, in_p)
print(sql)

df_wkago = pd.read_sql(sql, const)
df_wkago.shape[0]


SELECT name, price AS price_w
FROM price 
WHERE date = '2022-03-18' AND name IN ('AIMIRT', 'ASP', 'BCH', 'CPNCG', 'DIF', 'DOHOME', 'HREIT', 'IP', 'JASIF', 'KCE', 'MAKRO', 'MCS', 'NER', 'PTT', 'PTTGC', 'RATCH', 'RCL', 'RJH', 'SCC', 'SCCC', 'SINGER', 'SIS', 'SPALI', 'STA', 'SYNEX', 'TISCO', 'TMT', 'TPIPL', 'WHART') 
ORDER BY name


29

In [10]:
trend = pd.merge(df_today, df_wkago, on='name',how='inner')
trend.shape

(29, 3)

In [11]:
def performance(vals):
    price_d, price_w = vals
    if (price_d > price_w):
        return 'Better'
    elif (price_d < price_w):
        return 'Worse'
    else:
        return 'No Change'

In [12]:
trend['percent'] = (trend.price_d-trend.price_w)/trend.price_w * 100
trend["perform"] = trend[["price_d","price_w"]].apply(performance, axis=1)
trend.sort_values(by=['percent'],ascending=[True]).style.format(format_dict)

,name,price_d,price_w,percent,perform
16,RCL,43.75,50.50,-13.37%,Worse
24,SYNEX,26.00,27.75,-6.31%,Worse
5,DOHOME,21.30,22.60,-5.75%,Worse
10,MAKRO,39.50,41.25,-4.24%,Worse
23,STA,26.75,27.25,-1.83%,Worse
27,TPIPL,1.64,1.67,-1.80%,Worse
11,MCS,13.90,14.10,-1.42%,Worse
19,SCCC,153.50,155.00,-0.97%,Worse
17,RJH,32.00,32.25,-0.78%,Worse
0,AIMIRT,12.80,12.90,-0.78%,Worse


In [13]:
trend.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Worse,44.83%
Better,37.93%
No Change,17.24%


In [14]:
sql = '''
SELECT name, max_price AS max, min_price AS min, pe, pbv, daily_volume AS volume, beta, market
FROM stocks
'''
my_stocks = pd.read_sql(sql, conmy)
my_stocks.shape

(359, 8)

In [15]:
filters = [
   (my_stocks.market.str.contains('SET50')),
   (my_stocks.market.str.contains('SET100')),
   (my_stocks.market.str.contains('mai'))]
values = ['SET50','SET100','mai']
my_stocks["mrkt"] = np.select(filters, values, default='SET999')

In [16]:
trend2 = pd.merge(trend, my_stocks, on='name', how='inner')
trend2.sort_values(['percent'],ascending=[True]).shape

(29, 13)

In [17]:
set50 = trend2.mrkt.str.contains('SET50') 
flt_set50 = trend2[set50]
flt_set50[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt
13,PTT,39.00,39.00,0.00%,No Change,SET50
15,RATCH,44.00,44.00,0.00%,No Change,SET50
18,SCC,381.00,381.00,0.00%,No Change,SET50
25,TISCO,99.75,99.50,0.25%,Better,SET50
9,KCE,65.00,63.75,1.96%,Better,SET50
14,PTTGC,51.50,50.50,1.98%,Better,SET50


In [18]:
flt_set50.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,50.00%
No Change,50.00%


In [19]:
set100 = trend2.mrkt.str.contains('SET100')
flt_set100 = trend2[set100]
flt_set100[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt
16,RCL,43.75,50.50,-13.37%,Worse,SET100
24,SYNEX,26.00,27.75,-6.31%,Worse,SET100
5,DOHOME,21.30,22.60,-5.75%,Worse,SET100
23,STA,26.75,27.25,-1.83%,Worse,SET100
22,SPALI,22.00,21.90,0.46%,Better,SET100
2,BCH,21.40,21.20,0.94%,Better,SET100
20,SINGER,51.50,49.50,4.04%,Better,SET100


In [20]:
flt_set100[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Worse,57.14%
Better,42.86%


In [21]:
set999 = trend2.mrkt.str.contains('SET999')
flt_set999 = trend2[set999]
flt_set999[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt
10,MAKRO,39.50,41.25,-4.24%,Worse,SET999
27,TPIPL,1.64,1.67,-1.80%,Worse,SET999
11,MCS,13.90,14.10,-1.42%,Worse,SET999
19,SCCC,153.50,155.00,-0.97%,Worse,SET999
17,RJH,32.00,32.25,-0.78%,Worse,SET999
0,AIMIRT,12.80,12.90,-0.78%,Worse,SET999
21,SIS,38.25,38.50,-0.65%,Worse,SET999
1,ASP,3.46,3.48,-0.57%,Worse,SET999
3,CPNCG,12.30,12.30,0.00%,No Change,SET999
12,NER,7.10,7.10,0.00%,No Change,SET999


In [22]:
flt_set999[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Worse,53.33%
Better,33.33%
No Change,13.33%


In [23]:
setmai = trend2.mrkt.str.contains('mai')
flt_setmai = trend2[setmai]
flt_setmai[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt
7,IP,19.00,19.10,-0.52%,Worse,mai


In [24]:
flt_setmai[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Worse,100.00%
